# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 




### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [60]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
import re

In [61]:
pip install livelossplot

In [62]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [63]:
dataTraining.iloc[:10]

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6
9784,1943,The Ox-Bow Incident,two drifters are passing through a western tow...,"['Drama', 'Western']",8.1
8804,2004,Hellbent,two gay men on a date are murdered the night b...,"['Horror', 'Thriller']",5.5
3501,1941,How Green Was My Valley,adult huw morgan is about to move away from th...,"['Drama', 'Family']",7.8
7145,2008,Ghost Town,bertram pincus is a man whose people skills le...,"['Comedy', 'Drama', 'Fantasy', 'Romance']",6.7
4067,1996,Kingpin,roy munson was raised to be the best bowler in...,"['Comedy', 'Sport']",6.9


#### Arreglo a la data

In [64]:
def arregla_doble_espacio(sentence):
    cleaned_sentence = re.sub(r" '","'", sentence)
    return cleaned_sentence

def arregla_espacio2(sentence):
    cleaned_sentence = re.sub(r"' ","'", sentence)
    return cleaned_sentence

def arregla_abrev(sentence):
    cleaned_sentence = re.sub(r"won't"," will not", sentence)
    cleaned_sentence = re.sub(r"can\'t"," can not", cleaned_sentence)
    cleaned_sentence = re.sub(r"n\'t"," not", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'re"," are", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'s"," is", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'d"," would", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'ll"," will", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'t"," not", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'ve"," have", cleaned_sentence)
    cleaned_sentence = re.sub(r"\'m"," am", cleaned_sentence)
    return cleaned_sentence

def remove_punctuation(sentence):
    cleaned_sentence = re.sub(r'[?|!|\'|"|#]', '', sentence)
    cleaned_sentence = re.sub(r'[,|.|;|:|(|)|{|}|\|/|<|>]|-', ' ', cleaned_sentence)
    cleaned_sentence = cleaned_sentence.replace("\n"," ")
    cleaned_sentence = re.sub(' +', ' ', cleaned_sentence)
    return cleaned_sentence

In [65]:
dataTraining['plot'] = dataTraining['plot'].apply(arregla_doble_espacio)
dataTraining['plot'] = dataTraining['plot'].apply(arregla_espacio2)
dataTraining['plot'] = dataTraining['plot'].apply(arregla_abrev)
dataTraining['plot'] = dataTraining['plot'].apply(remove_punctuation)

In [66]:
dataTesting['plot'] = dataTesting['plot'].apply(arregla_doble_espacio)
dataTesting['plot'] = dataTesting['plot'].apply(arregla_espacio2)
dataTesting['plot'] = dataTesting['plot'].apply(arregla_abrev)
dataTesting['plot'] = dataTesting['plot'].apply(remove_punctuation)

In [67]:
#expotar la base arreglada para revisar en excel
#dataTraining.to_csv('train.csv', index_label='ID')

### Create y

In [68]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
#y_genres = dataTraining['genres']

In [69]:
dataTesting.iloc[:10]

,year,title,plot
1,1999,Message in a Bottle,who meets by fate shall be sealed by fate ther...
4,1978,Midnight Express,the true story of billy hayes an american coll...
5,1996,Primal Fear,martin vail left the chicago da is office to b...
6,1950,Crisis,husband and wife americans dr eugene and mrs h...
7,1959,The Tingler,the coroner and scientist dr warren chapin is ...
9,1961,Barabbas,epic account of the thief barabbas who was spa...
11,2005,Rize,reveals a groundbreaking dance phenomenon that...
13,2000,The Yards,in the rail yards of queens contractors repair...
16,1992,Used People,at her husband is funeral pearl shirley maclai...
21,2004,My Baby's Daddy,lonnie dominic and g are three bachelor buddie...


In [70]:
lista =dataTraining['plot']
llen=list()
for l in lista:
  llen.append(len(l.split()))
pd.Series(llen).describe(percentiles=[0.8,0.9,0.95,0.975])

count    7895.000000
mean      126.632299
std        84.422392
min         1.000000
50%       110.000000
80%       180.000000
90%       240.000000
95%       290.300000
97.5%     334.000000
max      1723.000000
dtype: float64

In [61]:
max_words = 340    #340
batch_size = 32
nb_epoch = 10

## **MLP**


In [112]:
X=dataTraining['plot']
vect = CountVectorizer(stop_words='english', max_features=10000)
X_dtm = vect.fit_transform(X)
words = list(vect.vocabulary_.keys())
vocabulary = {x: idx + 1 for idx, x in enumerate(set(words))}

In [113]:
lp=list()
for p in X:
    lidx=list()
    #print(p)
    listapalabras=p.split()
    #print(listapalabras)
    for w in listapalabras:
      if w in vocabulary.keys():
        lidx.append(vocabulary[w])
    lp.append(lidx)


In [117]:
max_words =500
from keras.preprocessing import sequence
X_pad2 = sequence.pad_sequences(lp, maxlen=max_words)
len(X_pad2)
X_pad2

array([[   0,    0,    0, ..., 6028,  852, 1360],
       [   0,    0,    0, ..., 8062, 3746, 9442],
       [   0,    0,    0, ..., 7776, 3529, 1146],
       ...,
       [   0,    0,    0, ..., 1020, 3739, 6015],
       [   0,    0,    0, ..., 7347,  766, 1873],
       [   0,    0,    0, ..., 6554, 5575, 9589]], dtype=int32)

In [119]:
#Divido la base de datos
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_pad2, y_genres, test_size=0.33, random_state=42)


In [120]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras import layers
model = Sequential()
model.add(layers.Embedding(input_dim=len(vocabulary) + 1, output_dim=400,   input_length=X_pad2.shape[1]))  #len(vocabulary) + 1
model.add(layers.Flatten())
model.add(layers.Dense(500,activation='relu')) #500
#model.add(Dropout(0.1)) #0.2
model.add(layers.Dense(500,activation='relu')) #500
model.add(layers.Dense(500,activation='relu')) #500
#model.add(layers.Dense(300,activation='relu')) #500
model.add(layers.Dense(24, activation='softmax'))
metric = tf.keras.metrics.AUC(multi_label = True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metric])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 500, 400)          4000400   
_________________________________________________________________
flatten_23 (Flatten)         (None, 200000)            0         
_________________________________________________________________
dense_50 (Dense)             (None, 500)               100000500 
_________________________________________________________________
dense_51 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_52 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_53 (Dense)             (None, 24)                12024     
Total params: 104,513,924
Trainable params: 104,513,924
Non-trainable params: 0
_______________________________________

In [87]:
model.fit(X_train, y_train_genres,
          validation_data = (X_test, y_test_genres),
          epochs=10, verbose=2,  batch_size=20) #50

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5289 samples, validate on 2606 samples
Epoch 1/10
 - 32s - loss: 0.3461 - auc_8: 0.4990 - val_loss: 0.3306 - val_auc_8: 0.5681
Epoch 2/10
 - 32s - loss: 0.2910 - auc_8: 0.6397 - val_loss: 0.3154 - val_auc_8: 0.6900
Epoch 3/10
 - 32s - loss: 0.2256 - auc_8: 0.7350 - val_loss: 0.3000 - val_auc_8: 0.7679
Epoch 4/10
 - 32s - loss: 0.1931 - auc_8: 0.7933 - val_loss: 0.3052 - val_auc_8: 0.8115
Epoch 5/10
 - 32s - loss: 0.1704 - auc_8: 0.8277 - val_loss: 0.3314 - val_auc_8: 0.8399
Epoch 6/10
 - 32s - loss: 0.1592 - auc_8: 0.8495 - val_loss: 0.3633 - val_auc_8: 0.8565
Epoch 7/10
 - 32s - loss: 0.1509 - auc_8: 0.8622 - val_loss: 0.3405 - val_auc_8: 0.8681
Epoch 8/10
 - 32s - loss: 0.1448 - auc_8: 0.8729 - val_loss: 0.3613 - val_auc_8: 0.8768
Epoch 9/10
 - 32s - loss: 0.1403 - auc_8: 0.8803 - val_loss: 0.3645 - val_auc_8: 0.8832
Epoch 10/10
 - 32s - loss: 0.1380 - auc_8: 0.8859 - val_loss: 0.3695 - val_auc_8: 0.8881


In [88]:
y_pred_genres = model.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro') #82.8

0.8153135420043781

### Predict the testing dataset

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [ ]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [ ]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.096,0.077,0.017,0.020,0.391,0.077,0.013,0.504,0.046,0.133,...,0.076,0.063,0.000,0.477,0.023,0.036,0.014,0.210,0.029,0.017
4,0.149,0.035,0.020,0.126,0.320,0.346,0.123,0.499,0.038,0.034,...,0.019,0.034,0.000,0.106,0.012,0.008,0.005,0.190,0.086,0.015
5,0.100,0.037,0.003,0.020,0.170,0.532,0.006,0.677,0.029,0.039,...,0.001,0.258,0.000,0.210,0.046,0.000,0.007,0.430,0.020,0.039
6,0.162,0.084,0.009,0.025,0.280,0.074,0.017,0.603,0.062,0.034,...,0.057,0.099,0.001,0.232,0.120,0.001,0.031,0.319,0.069,0.012
7,0.105,0.094,0.010,0.022,0.209,0.271,0.010,0.389,0.040,0.098,...,0.022,0.106,0.000,0.134,0.324,0.000,0.007,0.217,0.012,0.006


In [ ]:
res.to_csv('pred_genres_text_RF.csv', index_label='ID')